# Scientific visualisation in Jupyter notebooks

As a bridge to the next subject of the visualisation course, scientific visualisation, this notebook will give you a couple of examples of the possibilities of scientific visualisations in the Jupyter notebooks.

## Ipyvolume

Ipyvolume makes it possible to make 3D interactive jupyter-notebook widgets for volume and mesh rendering in the Jupyter notebooks.

In [1]:
import numpy as np
import ipyvolume as ipv
import vtk
from vtk.util import numpy_support as vn
from vtk.util.numpy_support import vtk_to_numpy

# Read the mummy volume dataset from the vtk file with a vtk data reader
reader = vtk.vtkDataSetReader()
reader.SetFileName("./data/volumes/mummy.128.vtk");
reader.Update()

# extract the vtk image data to a numpy array
imdata = reader.GetOutput()
pdata = imdata.GetPointData()
mummy = vn.vtk_to_numpy(pdata.GetArray('scalars'))

# formate the data from 0 to 1 and change the shape to a volume
mummy = mummy.astype(float)
mummy = (mummy - mummy.min()) / mummy.ptp()
mummy = mummy.reshape(128,128,128)

# extract a iso suface from the volume
contourFilter = vtk.vtkContourFilter();
contourFilter.SetInputConnection(reader.GetOutputPort());
contourFilter.GenerateValues(1, 127, 127); # (numContours, rangeStart, rangeEnd)
contourFilter.Update();

# Get poly data
polydata = contourFilter.GetOutput();

# Make a ipyvolume figure in which to draw the 3D visualisations
fig = ipv.figure(width=500, height=500, lighting=True, controls=True)

# Get vertex locations and store x, y ,z components in seperate arrays
vertices = vtk_to_numpy(polydata.GetPoints().GetData())
vx,vy,vz = np.transpose(vertices)

# extract the triangle data and remove the vertex count per triangle
polys = vtk_to_numpy(polydata.GetPolys().GetData())
polys = np.delete(polys, slice(None, None, 4))
polys = polys.reshape((polys.shape[0]//3,3))

# create a mesh widget
mesh = ipv.plot_trisurf(vx, vy, vz, triangles=polys, color='orange')

# create a volume widget
volume = ipv.volshow(mummy, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1, extent=[(0,239.522),(0,239.522),(0,242.951)])

# draw the widgets
ipv.show()

/projects/jhssrf019/JHS_installations/Python/lib/python3.10/site-packages/ipyvolume/serialize.py:102: RuntimeWarning: invalid value encountered in cast
  subdata[..., i] = ((gradient[i][zindex] / 2.0 + 0.5) * 255).astype(np.uint8)


Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.25, max=1.0, ste…

As you can see it shows both the volume and the mesh widget in one figure. If you want to show one or the other you can disable the visibility within the python code cells (see below) or you can make interactive widgets for them like the sliders above the visualisations, for more information on the interactive widgets can be found [here](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html).

In [2]:
mesh.visible = False